# Online Pipeline

In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
from sklearn.cluster import DBSCAN
from sklearn import metrics
import numpy as np
from torchvision import transforms
import torch
import clip
import matplotlib.pyplot as plt
import pickle
from PIL import Image
from rich import print
# to_pil = transforms.ToPILImage()


def find_avg_centroid(x):
    return np.mean(x, axis=0)

# def find_avg_centroid(x):
#     length, dim = x.shape
#     return np.array([np.sum(x[:, i])/length for i in range(dim)])

def find_nearest_centroids(x, y):
    if not isinstance(x, (np.ndarray, np.generic)):
        x = x.detach().cpu().numpy()
        
    clf = NearestCentroid(metric='manhattan')
    clf = clf.fit(x, y)
    centroids = clf.centroids_
    return centroids

def find_knn(x, centroids, n_neighbors=10):
    if not isinstance(x, (np.ndarray, np.generic)):
        x = x.detach().cpu().numpy()

    knn_search = NearestNeighbors(n_neighbors=n_neighbors,
                            # metric='cosine', # because CLIP model
                            metric='manhattan', # because CLIP model
                            algorithm='auto', #'ball_tree'
                            n_jobs=-1)  
    knn_search.fit(x)

    indexes = knn_search.kneighbors(centroids, return_distance=False)
    return x[indexes][0]

        
def get_clusters(x):
    # X = mde_embedding
    # x = x.detach().cpu().numpy()
    # db = DBSCAN(eps=0.8, min_samples=10, n_jobs=-1, metric='manhattan').fit(X)
    # db = OPTICS(min_samples=2).fit(x)
    db = DBSCAN(eps=0.3, min_samples=10, leaf_size=30, 
                n_jobs=-1, metric='cosine').fit(x)

    # core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    # core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print("Estimated number of clusters: %d" % n_clusters_)
    print("Estimated number of noise points: %d" % n_noise_)
    if n_clusters_ == 0:
        return x, labels, n_clusters_
    # Remove outliers to avoid affecting the centroids
    x = x[np.where(labels != -1)]
    labels = labels[np.where(labels != -1)]
    
    return x, labels, n_clusters_


def get_nearest_embedding_centroids(x, n_neighbors=10):
    # print(task.task.title())
    # print(f"CLIP Embedding shape: {task.embeddings.shape}")
    
    #1. Get the CLIP embeddings
    if not isinstance(x, (np.ndarray, np.generic)):
        x = x.detach().cpu().numpy()
    
    #3. Cluster the embeddings
    x, labels, n_clusters = get_clusters(x)

    #4. Find centroids of embeddings
    if n_clusters < 2:
        centroids = find_avg_centroid(x)
        centroids = centroids.reshape(1, -1)
    else:
        centroids = find_nearest_centroids(x, labels)

    # If some infinite/nan values appear
    centroids = np.nan_to_num(centroids)
    #5. Find knn of centroids        
    x_nearest_centroids = find_knn(x, centroids, n_neighbors)
    # x_nearest_centroids2 = np.vstack([find_knn(x, [centroid], n_neighbors) for centroid in centroids])
    # return centroids
    x_chosen = np.vstack([centroids, x_nearest_centroids])
    # x_chosen = np.vstack([centroids, x_nearest_centroids, x_nearest_centroids2])
    print(f"# nearest centroids ({round((x_chosen.shape[0]/x.shape[0])*100, 2)}%):", x_chosen.shape)
    print()

    return x_chosen


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: ", device)
embeder, preprocess = clip.load("ViT-B/32", device=device)

Device:  cpu

In [3]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util
from functools import lru_cache

@lru_cache(maxsize=None)
def get_embeddings(dataset):
    
    all_features = []
    all_labels = []
    all_tasks = []
    
    with torch.no_grad():
        for images, labels, tasks in tqdm(DataLoader(dataset, batch_size=100, num_workers=8)):
            features = embeder.encode_image(images.to(device))
            # features = embeder_resnet(images.to(device)).squeeze()

            all_features.append(features)
            all_labels.append(labels)
            all_tasks.append(tasks)

    return (torch.cat(all_features).cpu().numpy(), 
            torch.cat(all_labels).cpu().numpy(), torch.cat(all_tasks).cpu().numpy())


In [4]:
import sys
sys.path.append('./CLAD/')

In [5]:
import clad
import torchvision

root = "./data"
original_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.3252, 0.3283, 0.3407), (0.0265, 0.0241, 0.0252))

])

# all_train_sets = clad.get_cladc_train(root, transform=lambda x: x)
all_train_sets = clad.get_cladc_train(root, transform=original_transform)
# val_sets = clad.get_cladc_val(root, transform=torchvision.transforms.ToTensor())
print(len(all_train_sets))
print([len(ts) for ts in all_train_sets])

[INFO] No Detectron installation found, continuing without.


6

[5157, 1154, 6742, 2560, 4517, 2119]

In [6]:
# method_transform = torchvision.transforms.Compose([
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.ToPILImage()
# ])
method_transform = torchvision.transforms.Compose([
    preprocess
])
all_train_sets_method = clad.get_cladc_train(root, transform=method_transform)
print(len(all_train_sets_method))
print([len(ts) for ts in all_train_sets_method])

6

[5157, 1154, 6742, 2560, 4517, 2119]

In [7]:
import torch
from torch.utils.data import random_split

# Define the split sizes for each dataset
train_size = 0.8
# test_size = 0.2

# Split each dataset in the train_sets list into training and testing sets
train_sets = []
test_sets = []
for dataset in all_train_sets:
    dataset_size = len(dataset)
    train_dataset_size = int(train_size * dataset_size)
    test_dataset_size = dataset_size - train_dataset_size
    train_dataset, test_dataset = random_split(dataset, [train_dataset_size, test_dataset_size], 
                                               generator=torch.Generator().manual_seed(42))
    train_sets.append(train_dataset)
    test_sets.append(test_dataset)

print("Original data")
print('Training lenghts: ', [len(ts) for ts in train_sets])
print('Testing lenghts: ', [len(ts) for ts in test_sets])

train_sets_method = []
test_sets_method = []
for dataset in all_train_sets_method:
    dataset_size = len(dataset)
    train_dataset_size = int(train_size * dataset_size)
    test_dataset_size = dataset_size - train_dataset_size
    train_dataset, test_dataset = random_split(dataset, [train_dataset_size, test_dataset_size], 
                                               generator=torch.Generator().manual_seed(42))
    train_sets_method.append(train_dataset)
    test_sets_method.append(test_dataset)

print("Method data")
print('Training lenghts: ', [len(ts) for ts in train_sets_method])
print('Testing lenghts: ', [len(ts) for ts in test_sets_method])

Original data

Training lenghts: 
[4125, 923, 5393, 2048, 3613, 1695]

Testing lenghts: 
[1032, 231, 1349, 512, 904, 424]

Method data

Training lenghts: 
[4125, 923, 5393, 2048, 3613, 1695]

Testing lenghts: 
[1032, 231, 1349, 512, 904, 424]

In [8]:
%%time
from torch.utils.data.dataset import TensorDataset
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.benchmarks.utils import make_classification_dataset
from torch.utils.data import DataLoader
from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark



# Original benchmark with tasks
training_datasets = list()
testing_datasets = list()

for task, (train_s, test_s) in enumerate(zip(train_sets, test_sets), start=0):
    training_datasets.append(make_classification_dataset(train_s, task_labels=np.repeat(task, len(train_s))))
    testing_datasets.append(make_classification_dataset(test_s, task_labels=np.repeat(task, len(test_s))))

benchmark= dataset_benchmark(
    training_datasets,
    testing_datasets
)

# Original benchmark without tasks
training_datasets = list()
testing_datasets = list()
for task, (train_s, test_s) in enumerate(zip(train_sets, test_sets), start=0):
    training_datasets.append(make_classification_dataset(train_s, task_labels=np.repeat(0, len(train_s))))
    testing_datasets.append(make_classification_dataset(test_s, task_labels=np.repeat(0, len(test_s))))

benchmark_no_task= dataset_benchmark(
    training_datasets,
    testing_datasets
)

# Method benchmark
training_datasets = list()
testing_datasets = list()
for task, (train_s, test_s) in enumerate(zip(train_sets_method, test_sets_method), start=0):
    training_datasets.append(make_classification_dataset(train_s, task_labels=np.repeat(task, len(train_s))))
    testing_datasets.append(make_classification_dataset(test_s, task_labels=np.repeat(task, len(test_s))))

benchmark_method= dataset_benchmark(
    training_datasets,
    testing_datasets
)

CPU times: user 511 ms, sys: 147 ms, total: 659 ms
Wall time: 712 ms


## Data preparation

In [9]:
train_stream = benchmark_method.train_stream
test_stream = benchmark_method.test_stream

all_x_train_emb = list()
all_y_train = list()
all_t_train = list()

all_x_test_emb = list()
all_y_test = list()
all_t_test = list()


for task, experience in enumerate(train_stream):
    print("Getting embeddings")
    x_train_emb, y_train, t_train = get_embeddings(train_stream[task].dataset)

    print("x_train shape: ", x_train_emb.shape)
    all_x_train_emb.append(x_train_emb)
    all_y_train.append(y_train)
    all_t_train.append(t_train)

    x_test_emb, y_test, t_test = get_embeddings(test_stream[task].dataset)
    print("x_test shape: ", x_test_emb.shape)
    all_x_test_emb.append(x_test_emb)
    all_y_test.append(y_test)
    all_t_test.append(t_test)

Getting embeddings

100%|██████████| 42/42 [00:33<00:00,  1.25it/s]


x_train shape: 
(4125, 512)

100%|██████████| 11/11 [00:09<00:00,  1.13it/s]


x_test shape: 
(1032, 512)

...

Getting embeddings

100%|██████████| 17/17 [00:14<00:00,  1.15it/s]


x_train shape: 
(1695, 512)

100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


x_test shape: 
(424, 512)

## Pipeline

In [ ]:
from dataclasses import dataclass

@dataclass
class Memory:
    task: int
    shape: List = None
    centroids: InitVar[List] = None
    def __post_init__(self, points):
        self.centroids = points
        self.shape = self.centroids.shape

def online_pipeline(memory, task, x_new):
    decisions = list()

    # If Empty memory -> save points and use model
    if not memory:  
        # print("Empty memory")
        return ['save_centroids', 'use_head']
    
    drift_history = []
    for index, x_ref in enumerate(memory[::-1], start=1): # loop in reversed order
        # print(f"Calculating drift between Task(current={task}, memory={x_ref.task})")
        drift = calculate_drift(x_ref.centroids, x_new)
        is_drift = drift['data']['is_drift']

        
        # Case when the next task is the same as before (two consecutive tasks are the same)
        if (index == 1) and (not is_drift):  
            return ['use_head']  
        
        drift_history.append(is_drift)

    # all(drift(task, each element in memory)) = True, that is, not known task
    if all(drift_history):  
        return ['save_centroids', 'train_task_classifier', 'add_head', 'use_head']
    else: # any(drift(task, each element in memory)) = False, that is, known task
        return ['classify_task', 'use_head']

for task in enumerate(sequence):
    x_new = get_nearest_embedding_centroids(train_dict[task], n_neighbors=10)
    decisions = online_pipeline(memory, task, x_new)
    task_id = task
    for decision in decisions:
        if decision == 'save_centroids':  # save new points in memory
            memory.append(Memory(task=task, centroids=x_new))
        elif decision == 'train_task_classifier':  # train the task classifier because there is drift
            x_train = [m.centroids for m in memory]
            y_train = [np.repeat(m.task, m.shape[0]) for m in memory]
            x_train = np.vstack(x_train)
            y_train = np.hstack(y_train)
            classifier = get_classifier(x_train, y_train)
            task_id = memory[-1].task # last task_id
        elif decision == 'classify_task':  # classify the new points using the task classifier
            task_id = classifier.predict(x_new)
            task_id = stats.mode(task_id, keepdims=True) # takes the mode of the predictions
            task_id = int(task_id[0])
        elif decision == 'add_head':  # add new head to current multi-head model
            models = add_head_model(models, x_new, y_new, task_id) # Add head based on task_id
        elif decision == 'use_head':  # use current head based on task_id
            use_model_head(models, x_val, y_val, task_id)
